Imports

In [57]:
from time import sleep
import pickle, json, re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


from webdriver_manager.chrome import ChromeDriverManager

from word2number import w2n

WebDriver Initialization

In [58]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()), options=options
)

Location

In [59]:
driver.get("https://www.swiggy.com/city/delhi")

Scroll to the Bottom

In [9]:
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.find_element(By.CLASS_NAME, "fBowAU").click()
    # driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script(
        "window.scrollTo({top: document.body.scrollHeight-3000,left: 0,behavior: 'smooth'});"
    )

    # Wait to load page
    sleep(scroll_pause_time)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    sleep(1)

Create Restraunt's Links List

In [10]:
restraunt_elements_list = driver.find_elements(
    by=By.CSS_SELECTOR, value=".jXGZuP a")
restraunt_links_list = []
for restraunt_element in restraunt_elements_list:
    if (
        restraunt_element.get_attribute("href") != "https://www.swiggy.com/search"
        and restraunt_element.get_attribute("href") != None
    ):
        restraunt_links_list.append(restraunt_element.get_attribute("href"))

print(f"Restraunts Found: {len(restraunt_links_list)}")

with open("restraunt_links_list.log", "wb") as file1:
    pickle.dump(restraunt_links_list, file1)

Restraunts Found: 1362


Verify Restraunt's Links List

In [26]:
with open("restraunt_links_list.log", "rb") as file1:
    restraunt_links_list = pickle.load(file1)
print(f"Links Saved: {len(restraunt_links_list)}")
data = []

Links Saved: 1362


Collect Data

In [63]:
for restraunt_number in range(1244, len(restraunt_links_list)):
    restraunt_data = {}

    driver.get(restraunt_links_list[restraunt_number])

    restraunt_data["restraunt_name"] = driver.find_element(by=By.CLASS_NAME, value="RestaurantNameAddress_name__2IaTv").text
    
    restraunt_data["restraunt_category"] = (
        driver.find_element(
            by=By.CLASS_NAME, value="RestaurantNameAddress_cuisines__mBHr2").text
    ).split(", ")

    title_string = driver.find_element(
        by=By.CSS_SELECTOR, value="head meta[name=og_title]"
    ).get_attribute("content")
    restraunt_data["restraunt_address"] = re.search(r'\|([^|]+)$', title_string).group(1).strip()

    restraunt_data["rating"] = driver.find_element(
        by=By.CSS_SELECTOR, value=".RestaurantRatings_avgRating__1TOWY span:nth-child(2)"
    ).text

    restraunt_data["no_of_ratings"] = driver.find_element(
        by=By.CLASS_NAME, value="RestaurantRatings_totalRatings__3d6Zc").text

    price_string = driver.find_element(
        by=By.CSS_SELECTOR, value=".RestaurantTimeCost_item__2HCUz:nth-child(2) span").text
    restraunt_data["average_cost"] = int(int(re.search(r'₹(\d+)', price_string).group(
        1).strip())/w2n.word_to_num(re.search(r'(\w+)$', price_string).group(1).strip()))
    data.append(restraunt_data)

Store Data to JSON File

In [62]:
json_data = json.dumps(data, indent=4)
with open("data1.json", "w") as file2:
    json.dump(data, file2)

In [64]:
print(len(data))

1267


Quit WebDriver

In [ ]:
driver.quit()